In [ ]:
%matplotlib widget

import scipp as sc
import plopp as pp
import numpy as np
from plopp import widgets
import ipywidgets as ipw
from scipp.scipy.ndimage import gaussian_filter

In [ ]:
da = sc.io.load_hdf5('nyc_taxi_data_2015_small.h5')
data = da.group("dropoff_hour").hist(dropoff_latitude=500, dropoff_longitude=500)
data

In [ ]:
# Slider node that provides index to slice
slider = ipw.IntSlider(description="Hour:", min=0, max=23)
slider_node = pp.widget_node(slider)

# Node that actually does the slicing
slice_node = pp.Node(lambda da, ind: da["dropoff_hour", ind], da=data, ind=slider_node)

# Add a 2D figure to show the NYC map
fig2d = pp.imagefigure(slice_node, norm="log", cbar=False)

# Add a node after the slicing to sum along the latitude dimension
sum_lat = pp.Node(sc.sum, slice_node, dim="dropoff_latitude")
# Add a node after the slicing to sum along the longitude dimension
sum_lon = pp.Node(sc.sum, slice_node, dim="dropoff_longitude")

# Add a node after the sum that performs as Gaussian smoothing
smooth = pp.Node(gaussian_filter, sum_lat, sigma=5)

# Add a 1D figure that will display sum along longitude
fig_lon = pp.linefigure(sum_lon, norm="log")
# Add a 1D figure that will display both raw latitude sum and smoothed data
fig_lat = pp.linefigure(sum_lat, smooth, norm="log")

widgets.Box([slider, fig2d, fig_lon, fig_lat])  # Container box

In [ ]:
# Slider node that provides index to slice
slider = ipw.IntSlider(description="Hour:", min=0, max=23)
slider_node = pp.widget_node(slider)

# Node that actually does the slicing
slice_node = pp.Node(lambda da, ind: da["dropoff_hour", ind], da=data, ind=slider_node)

# Add a 2D figure to show the NYC map
fig2d = pp.figure2d(slice_node, norm="log", cbar=False)

# Add a node after the slicing to sum along the latitude dimension
sum_lat = pp.Node(sc.sum, slice_node, dim="dropoff_latitude")
# Add a node after the slicing to sum along the longitude dimension
sum_lon = pp.Node(sc.sum, slice_node, dim="dropoff_longitude")


# Add a new slider that will act as input to the Gaussian smoothing node
smooth_slider = ipw.IntSlider(description="kernel:", min=1, max=25)
smooth_slider_node = pp.widget_node(smooth_slider)


# Add a node after the sum that performs as Gaussian smoothing
smooth = pp.Node(gaussian_filter, sum_lat, sigma=smooth_slider_node)

# Add a 1D figure that will display sum along longitude
fig_lon = pp.figure1d(sum_lon, norm="log")
# Add a 1D figure that will display both raw sum and smoothed data
fig_lat = pp.figure1d(sum_lat, smooth, norm="log")

widgets.Box([[slider, smooth_slider], [fig2d, fig_lon], fig_lat])  # Container box

In [ ]:
pp.show_graph(slider_node)

In [ ]:
a = pp.Node(np.array([1, 2, 3, 4, 5]))
b = pp.Node(np.array([6, 7, 8, 9, 10]))